# Develop a light GAN for post processing
* The input must be the biggest size in the dataset
* Both Generator and Discriminator must fit in the GPU (48GB VRAM)


### Check biggest (384,384,384 although there is only 1 case with this shape...)

In [3]:
# Find the bigest shape
import os
import tifffile as tiff
import numpy as np
import nibabel as nib
all_shapes = []
root_dataset = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/Challenge_dataset_updated/train_labels_nii_crop"
for file_name in os.listdir(root_dataset):
    if file_name.endswith('.nii.gz'):
        file_path = os.path.join(root_dataset, file_name)
        img_array = nib.load(file_path).get_fdata()
        all_shapes.append(img_array.shape)
        #if img_array.shape[0]>320:
        print(img_array.shape)

socket.send() raised exception.
socket.send() raised exception.


Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x769b37980890>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 769b34361690, raw_cell="# Find the bigest shape
import os
import tifffile .." transformed_cell="# Find the bigest shape
import os
import tifffile .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/notebooks/11_GANs_postprocess.ipynb#W2sZmlsZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

(314, 297, 320)
(314, 314, 320)
(250, 250, 256)
(314, 314, 320)
(250, 250, 256)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(250, 250, 256)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 314, 320)
(314, 31

socket.send() raised exception.
socket.send() raised exception.


(314, 314, 320)
(314, 314, 320)
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x769b37980890>> (for post_run_cell), with arguments args (<ExecutionResult object at 769b34360150, execution_count=3 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 769b34361690, raw_cell="# Find the bigest shape
import os
import tifffile .." transformed_cell="# Find the bigest shape
import os
import tifffile .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/notebooks/11_GANs_postprocess.ipynb#W2sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [10]:
x_all_shapes = [shape_element[0] for shape_element in all_shapes]
y_all_shapes = [shape_element[1] for shape_element in all_shapes]
z_all_shapes = [shape_element[2] for shape_element in all_shapes]

print(f"Biggest x: {max(x_all_shapes)}")
print(f"Biggest y: {max(y_all_shapes)}")
print(f"Biggest z: {max(z_all_shapes)}")

Biggest x: 384
Biggest y: 384
Biggest z: 384


### Create GANs

#### Generator

In [1]:
import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint

class Generator(torch.nn.Module):
    def __init__(self, in_channels=2, first_channels=16, out_channels=1, use_checkpointing=True):
        super(Generator, self).__init__()
        self.in_channels = in_channels
        self.first_channels = first_channels
        self.out_channels = out_channels
        self.use_checkpointing = use_checkpointing
        
        # ----------------------
        # ENCODER
        # ----------------------
        self.enc1 = nn.Sequential(
            nn.Conv3d(self.in_channels, self.first_channels, kernel_size=5, stride=2, padding=2, bias=False),
            nn.InstanceNorm3d(self.first_channels),
            nn.LeakyReLU(inplace=True)
        ) 

        self.enc2 = nn.Sequential(
            nn.Conv3d(self.first_channels, self.first_channels*2, kernel_size=5, stride=2, padding=2, bias=False),
            nn.InstanceNorm3d(self.first_channels*2),
            nn.LeakyReLU(inplace=True)
        ) 

        # ----------------------
        # BOTTLENECK (Hybrid Dilated)
        # ----------------------
        # FIX: Added Dilation=4 to close larger holes.
        self.bottleneck = nn.Sequential(
            # 1. Standard
            nn.Conv3d(self.first_channels*2, self.first_channels*2, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm3d(self.first_channels*2),
            nn.LeakyReLU(inplace=True),
            
            # 2. Wide View (Dilation 2)
            nn.Conv3d(self.first_channels*2, self.first_channels*2, kernel_size=3, padding=2, dilation=2, bias=False),
            nn.InstanceNorm3d(self.first_channels*2),
            nn.LeakyReLU(inplace=True),

            # 3. Wider View (Dilation 4) <--- ADDED
            # Receptive field increases significantly without memory cost
            nn.Conv3d(self.first_channels*2, self.first_channels*2, kernel_size=3, padding=4, dilation=4, bias=False),
            nn.InstanceNorm3d(self.first_channels*2),
            nn.LeakyReLU(inplace=True),
            
            # 4. Consolidate
            nn.Conv3d(self.first_channels*2, self.first_channels*2, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm3d(self.first_channels*2)
        )

        # ----------------------
        # DECODER
        # ----------------------
        
        # Dec 2: It's okay to use Nearest here because Dec 1 will clean it up.
        self.dec2 = nn.Sequential(
            nn.Conv3d(self.first_channels*4, self.first_channels*2, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm3d(self.first_channels*2),
            nn.LeakyReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='nearest') 
        )

        # Dec 1: FIX -> Switched to TRILINEAR
        # Since we feed directly into a 1x1 Conv, 'nearest' would leave blocky artifacts.
        # Trilinear ensures smooth gradients at the final boundary.
        self.dec1 = nn.Sequential(
            nn.Conv3d(self.first_channels + self.first_channels*2, self.first_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm3d(self.first_channels),
            nn.LeakyReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='trilinear', align_corners=False) # <--- CHANGED
        )

        # ----------------------
        # OUTPUT
        # ----------------------
        # FIX: bias=True. 
        # Allows the network to learn a global "confidence shift" easily.
        self.out = nn.Sequential(
            nn.Conv3d(self.first_channels, self.out_channels, kernel_size=1, stride=1, padding=0, bias=True) 
        )
        
        # Zero Init for Residual Learning
        nn.init.constant_(self.out[0].weight, 0)
        nn.init.constant_(self.out[0].bias, 0) # Init the bias to 0 too

    def forward(self, x):
        def run_layer(layer, inp):
            if self.use_checkpointing and inp.requires_grad:
                return checkpoint(layer, inp, use_reentrant=False)
            return layer(inp)

        # Encoder
        x_enc1 = self.enc1(x)
        x_enc2 = run_layer(self.enc2, x_enc1)
        
        # Bottleneck
        x_bottleneck = run_layer(self.bottleneck, x_enc2)
        
        # Decoder 2
        cat_d2 = torch.cat([x_enc2, x_bottleneck], dim=1)
        x_dec2 = run_layer(self.dec2, cat_d2)
        
        # Decoder 1
        cat_d1 = torch.cat([x_enc1, x_dec2], dim=1)
        x_dec1 = run_layer(self.dec1, cat_d1)
        
        # Delta (Correction)
        delta_logits = self.out(x_dec1)
        
        # Input Logits (Assumption: Input Ch 1 is ALREADY logits)
        input_logits = x[:, 1:2, :, :, :]
        if self.training and input_logits.max() <= 1.0 and input_logits.min() >= 0.0:
            import warnings
            warnings.warn("Input Channel 1 looks like Probabilities (0-1), but the network expects Logits (-inf to inf). Residual learning will fail!")
        
        return input_logits + delta_logits

In [6]:
import os
import torch
import torch.nn as nn
import psutil
from torch.amp import autocast, GradScaler

def test_train_step():
    # 1. Setup Device
    device = torch.device('cuda')
    print(f"Running on {device}...")
    
    if device.type == 'cuda':
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

    # 2. Initialize Model, Optimizer, and Scaler (for AMP)
    generator = Generator(in_channels=2, first_channels=16, out_channels=1).to(device)
    optimizer = torch.optim.Adam(generator.parameters(), lr=1e-4)
    criterion = nn.MSELoss() # Simple loss for testing
    scaler = GradScaler()     # Necessary for stable AMP training

    # 3. Create Inputs and Target
    # Input: [B, 2, 320, 320, 320]
    input_tensor = torch.randn(1, 2, 320, 320, 320, device=device)
    # Target: [B, 1, 320, 320, 320] (the ideal segmentation/output)
    target = torch.randn(1, 1, 320, 320, 320, device=device)

    print(f"Starting training step with AMP...")


    # --- TRAINING STEP START ---
    optimizer.zero_grad()

    # Forward pass with AMP
    with autocast(device_type=device.type):
        output = generator(input_tensor)
        loss = criterion(output, target)

    # Backward pass with Scaler
    # This is where memory peaks!
    scaler.scale(loss).backward()
    
    # Optimizer Step
    scaler.step(optimizer)
    scaler.update()
    # --- TRAINING STEP END ---

    print(f"Step successful. Loss: {loss.item():.4f}")

    # 4. Measure Memory
    if device.type == 'cuda':
        peak_mem = torch.cuda.max_memory_allocated() / (1024 ** 3)
        print(f"Peak Training VRAM: {peak_mem:.2f} GiB")
    else:
        process = psutil.Process(os.getpid())
        print(f"System RAM used: {process.memory_info().rss / (1024 ** 3):.2f} GiB")

test_train_step()

"""
Running on cuda...
Starting training step with AMP...

--- Forward Pass Start ---
Input x: torch.Size([1, 2, 320, 320, 320])
x_enc1 (after stride 2): torch.Size([1, 16, 160, 160, 160])
x_enc2 (after stride 2): torch.Size([1, 32, 80, 80, 80])
x_bottleneck: torch.Size([1, 32, 80, 80, 80])
cat_d2 (enc2 + bottleneck): torch.Size([1, 64, 80, 80, 80])
x_dec2 (after upsample): torch.Size([1, 32, 160, 160, 160])
cat_d1 (enc1 + dec2): torch.Size([1, 48, 160, 160, 160])
x_dec1 (after upsample): torch.Size([1, 16, 320, 320, 320])
delta_logits: torch.Size([1, 1, 320, 320, 320])
input_logits (from channel 1): torch.Size([1, 1, 320, 320, 320])
Final output: torch.Size([1, 1, 320, 320, 320])
--- Forward Pass End ---

Step successful. Loss: 2.0002
Peak Training VRAM: 12.56 GiB
"""

Running on cuda...
Starting training step with AMP...
Step successful. Loss: 1.9999
Peak Training VRAM: 12.56 GiB


'\nRunning on cuda...\nStarting training step with AMP...\n\n--- Forward Pass Start ---\nInput x: torch.Size([1, 2, 320, 320, 320])\nx_enc1 (after stride 2): torch.Size([1, 16, 160, 160, 160])\nx_enc2 (after stride 2): torch.Size([1, 32, 80, 80, 80])\nx_bottleneck: torch.Size([1, 32, 80, 80, 80])\ncat_d2 (enc2 + bottleneck): torch.Size([1, 64, 80, 80, 80])\nx_dec2 (after upsample): torch.Size([1, 32, 160, 160, 160])\ncat_d1 (enc1 + dec2): torch.Size([1, 48, 160, 160, 160])\nx_dec1 (after upsample): torch.Size([1, 16, 320, 320, 320])\ndelta_logits: torch.Size([1, 1, 320, 320, 320])\ninput_logits (from channel 1): torch.Size([1, 1, 320, 320, 320])\nFinal output: torch.Size([1, 1, 320, 320, 320])\n--- Forward Pass End ---\n\nStep successful. Loss: 2.0002\nPeak Training VRAM: 12.56 GiB\n'

#### Discriminator

In [2]:
import os
import torch
import torch.nn as nn
from torch.nn.utils import spectral_norm
from torch.amp import autocast, GradScaler
import psutil

class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels=2, initial_filters=16):
        super(PatchDiscriminator, self).__init__()
        
        # Input: (B, 2, 320, 320, 320)
        # Channel 0: CT Scan
        # Channel 1: Predicted Logits (or Probability)
        
        # We use Spectral Norm on all layers.
        # It is the modern standard to prevent the Discriminator from becoming too strong too fast.
        
        # Layer 1: 320 -> 160
        self.conv1 = nn.Sequential(
            spectral_norm(nn.Conv3d(in_channels, initial_filters, kernel_size=4, stride=2, padding=1, bias=False)),
            nn.LeakyReLU(0.2, inplace=True)
        )

        # Layer 2: 160 -> 80
        self.conv2 = nn.Sequential(
            spectral_norm(nn.Conv3d(initial_filters, initial_filters*2, kernel_size=4, stride=2, padding=1, bias=False)),
            nn.InstanceNorm3d(initial_filters*2), # Norm helps stability
            nn.LeakyReLU(0.2, inplace=True)
        )

        # Layer 3: 80 -> 40
        self.conv3 = nn.Sequential(
            spectral_norm(nn.Conv3d(initial_filters*2, initial_filters*4, kernel_size=4, stride=2, padding=1, bias=False)),
            nn.InstanceNorm3d(initial_filters*4),
            nn.LeakyReLU(0.2, inplace=True)
        )

        # Layer 4: 40 -> 38 (Stride 1, just reducing FOV slightly)
        # We stop downsampling here to keep the receptive field focused.
        self.conv4 = nn.Sequential(
            spectral_norm(nn.Conv3d(initial_filters*4, initial_filters*8, kernel_size=4, stride=1, padding=1, bias=False)),
            nn.InstanceNorm3d(initial_filters*8),
            nn.LeakyReLU(0.2, inplace=True)
        )

        # Output Layer: 38 -> 37
        # Maps to a 1-channel grid of "Real/Fake" scores
        self.final = nn.Conv3d(initial_filters*8, 1, kernel_size=4, stride=1, padding=1)

    def forward(self, x):
        # x should be concatenation of [CT, Logits/Mask]
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        out = self.final(x)
        
        return out



In [6]:
def test_train_step():
    # 1. Setup Device
    if torch.cuda.is_available():
        device = torch.device('cuda')
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
    else:
        device = torch.device('cpu')
        
    print(f"Running on {device}...")

    # 2. Initialize Model, Optimizer, and Scaler (for AMP)
    discriminator = PatchDiscriminator(in_channels=2, initial_filters=16).to(device)
    optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-4)
    criterion = nn.MSELoss() # Least Squares GAN (LSGAN) loss
    scaler = GradScaler()     # Necessary for stable AMP training

    # 3. Create Inputs
    # Input: [B, 2, 320, 320, 320] -> (CT + Predicted_Mask)
    input_tensor = torch.randn(1, 2, 320, 320, 320, device=device)
    
    print(f"Starting training step with AMP...")

    # --- TRAINING STEP START ---
    optimizer.zero_grad()

    # Forward pass with AMP
    with autocast(device_type=device.type):
        output = discriminator(input_tensor)
        print(f"Discriminator Output Shape: {output.shape}")
        
        # CRITICAL FIX: Creates target based on ACTUAL output shape (e.g., 38x38x38)
        # We cannot use a 320x320x320 target for a PatchGAN output.
        target = torch.randn_like(output, device=device) 
        
        loss = criterion(output, target)

    # Backward pass with Scaler
    print("Computing Gradients...")
    scaler.scale(loss).backward()
    
    # Optimizer Step
    scaler.step(optimizer)
    scaler.update()
    # --- TRAINING STEP END ---

    print(f"Step successful. Loss: {loss.item():.4f}")

    # 4. Measure Memory
    if device.type == 'cuda':
        peak_mem = torch.cuda.max_memory_allocated() / (1024 ** 3)
        print(f"Peak Training VRAM: {peak_mem:.2f} GiB")
    else:
        process = psutil.Process(os.getpid())
        print(f"System RAM used: {process.memory_info().rss / (1024 ** 3):.2f} GiB")

if __name__ == "__main__":
    test_train_step()

Running on cuda...
Starting training step with AMP...
D Input: torch.Size([1, 2, 320, 320, 320])
D Layer 1: torch.Size([1, 16, 160, 160, 160])
D Layer 2: torch.Size([1, 32, 80, 80, 80])
D Layer 3: torch.Size([1, 64, 40, 40, 40])
D Layer 4: torch.Size([1, 128, 39, 39, 39])
D Output (Patch Grid): torch.Size([1, 1, 38, 38, 38])
Discriminator Output Shape: torch.Size([1, 1, 38, 38, 38])
Computing Gradients...
Step successful. Loss: 1.3487
Peak Training VRAM: 1.64 GiB


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp import autocast, GradScaler

# -------------------------------------------------------
# 1. Helper Loss: Tversky (Crucial for Holes/Bridges)
# -------------------------------------------------------
class TverskyLoss(nn.Module):
    def __init__(self, alpha=0.3, beta=0.7, smooth=1):
        super(TverskyLoss, self).__init__()
        self.alpha = alpha # False Positives (Bridges)
        self.beta = beta   # False Negatives (Holes) - Higher = Punishment for holes
        self.smooth = smooth

    def forward(self, logits, targets):
        # Apply Sigmoid because Generator outputs Logits
        probs = torch.sigmoid(logits)
        
        # Flatten
        probs = probs.view(-1)
        targets = targets.view(-1)
        
        # Calculate TF/FP/FN
        TP = (probs * targets).sum()    
        FP = ((1 - targets) * probs).sum()
        FN = (targets * (1 - probs)).sum()
       
        tversky = (TP + self.smooth) / (TP + self.alpha*FP + self.beta*FN + self.smooth)  
        return 1 - tversky

# -------------------------------------------------------
# 2. Setup (Run this once)
# -------------------------------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Models
generator = Generator(in_channels=2, first_channels=16, out_channels=1).to(device)
discriminator = PatchDiscriminator(in_channels=2, initial_filters=16).to(device)

# Optimizers
# Discriminator usually needs slightly lower LR or same as Generator in LSGAN
opt_g = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.999))
opt_d = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.5, 0.999))

# Losses
criterion_GAN = nn.MSELoss()            # LSGAN Loss (Real=1, Fake=0)
criterion_BCE = nn.BCEWithLogitsLoss()  # Pixel-wise Accuracy
criterion_Tversky = TverskyLoss(alpha=0.3, beta=0.7) # Topology

# Scaler for AMP
scaler = GradScaler()

# Hyperparameters
LAMBDA_SEG = 100.0  # Pixel/Dice loss is 100x more important than GAN loss
LAMBDA_ADV = 1.0    # GAN loss is just the "spice" to fix texture

# -------------------------------------------------------
# 3. The Loop
# -------------------------------------------------------
def train_one_epoch(dataloader, epoch_index):
    generator.train()
    discriminator.train()
    
    for batch_idx, (real_ct, coarse_logits, real_gt_mask) in enumerate(dataloader):
        
        # Move to GPU
        real_ct = real_ct.to(device)
        coarse_logits = coarse_logits.to(device)
        real_gt_mask = real_gt_mask.to(device)
        
        # Input to Generator: (CT, Coarse_Logits)
        g_input = torch.cat([real_ct, coarse_logits], dim=1)
        
        #################################################
        #  UPDATE DISCRIMINATOR
        #  Goal: Maximize log(D(x, y)) + log(1 - D(x, G(x, z)))
        #################################################
        opt_d.zero_grad()
        
        with autocast(device_type=device.type):
            # --- Train with REAL (CT + Ground Truth) ---
            # Note: Discriminator sees (CT, Mask) concatenation
            real_pair = torch.cat([real_ct, real_gt_mask], dim=1)
            
            d_real_pred = discriminator(real_pair)
            
            # Target for REAL is 1.0
            target_real = torch.ones_like(d_real_pred)
            loss_d_real = criterion_GAN(d_real_pred, target_real)
            
            # --- Train with FAKE (CT + Refined Prediction) ---
            # 1. Generate Fake
            refined_logits = generator(g_input)
            refined_probs = torch.sigmoid(refined_logits) # Convert logits to prob for D
            
            # 2. Detach! Crucial! 
            # We don't want to backprop into Generator yet.
            fake_pair = torch.cat([real_ct, refined_probs.detach()], dim=1)
            
            d_fake_pred = discriminator(fake_pair)
            
            # Target for FAKE is 0.0
            target_fake = torch.zeros_like(d_fake_pred)
            loss_d_fake = criterion_GAN(d_fake_pred, target_fake)
            
            # Combine
            loss_d = (loss_d_real + loss_d_fake) * 0.5

        # Backward D
        scaler.scale(loss_d).backward()
        scaler.step(opt_d)
        
        #################################################
        #  UPDATE GENERATOR
        #  Goal: Fool the Discriminator (D outputs 1 for fake)
        #        AND minimize segmentation error
        #################################################
        opt_g.zero_grad()
        
        with autocast(device_type=device.type):
            # 1. Adversarial Loss (Fool D)
            # We construct the pair again (this time WITH gradients)
            # Use probabilities for D input
            refined_probs_for_g = torch.sigmoid(refined_logits) 
            fake_pair_for_g = torch.cat([real_ct, refined_probs_for_g], dim=1)
            
            d_pred_for_g = discriminator(fake_pair_for_g)
            
            # Generator wants D to say "1.0" (Real)
            target_g = torch.ones_like(d_pred_for_g)
            loss_g_adv = criterion_GAN(d_pred_for_g, target_g)
            
            # 2. Segmentation Loss (Ground Truth Accuracy)
            # Compare refined_logits directly to real_gt_mask
            loss_g_bce = criterion_BCE(refined_logits, real_gt_mask)
            loss_g_tversky = criterion_Tversky(refined_logits, real_gt_mask)
            
            # Combine
            # loss_seg dominates (weight 100), loss_adv refines (weight 1)
            loss_g_total = (LAMBDA_SEG * (loss_g_bce + loss_g_tversky)) + (LAMBDA_ADV * loss_g_adv)

        # Backward G
        scaler.scale(loss_g_total).backward()
        scaler.step(opt_g)
        
        # Update Scaler once per batch
        scaler.update()

        # Logging
        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch_index}] Batch [{batch_idx}] "
                  f"Loss D: {loss_d.item():.4f} | "
                  f"Loss G Total: {loss_g_total.item():.4f} "
                  f"(Adv: {loss_g_adv.item():.4f}, Seg: {loss_g_bce.item():.4f})")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.amp import autocast, GradScaler
from torch.nn.utils import spectral_norm
def run_test():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Running test on: {device}")

    # A. Create Dummy Data
    # Batch=2, Size=64^3 (Small enough for CPU testing if needed)
    B, D, H, W = 2, 64, 64, 64
    
    # CT Scan (Normalized)
    ct_data = torch.randn(B, 1, D, H, W)
    # Coarse Logits (Simulated output from previous net)
    coarse_logits = torch.randn(B, 1, D, H, W)
    # Ground Truth (Binary Mask 0 or 1)
    gt_mask = torch.randint(0, 2, (B, 1, D, H, W)).float()

    dataset = TensorDataset(ct_data, coarse_logits, gt_mask)
    dataloader = DataLoader(dataset, batch_size=2)

    # B. Initialize Models & Optimizers
    gen = Generator().to(device)
    disc = PatchDiscriminator().to(device)
    
    opt_g = optim.Adam(gen.parameters(), lr=1e-3)
    opt_d = optim.Adam(disc.parameters(), lr=1e-3)
    
    criterion_GAN = nn.MSELoss()
    criterion_BCE = nn.BCEWithLogitsLoss()
    criterion_Tversky = TverskyLoss()
    scaler = GradScaler()

    LAMBDA_SEG = 100.0
    LAMBDA_ADV = 1.0

    print("Setup complete. Starting training loop...")

    # ==========================================
    # 3. The Training Loop Logic
    # ==========================================
    gen.train()
    disc.train()
    
    # Run 3 steps to verify gradients actually change
    initial_loss = None
    
    for step in range(3):
        for real_ct, coarse, real_gt in dataloader:
            real_ct, coarse, real_gt = real_ct.to(device), coarse.to(device), real_gt.to(device)
            
            # --- 1. Discriminator Step ---
            opt_d.zero_grad()
            with autocast(device_type=device.type):
                # Real
                d_input_real = torch.cat([real_ct, real_gt], dim=1)
                d_real = disc(d_input_real)
                loss_d_real = criterion_GAN(d_real, torch.ones_like(d_real))
                
                # Fake
                g_input = torch.cat([real_ct, coarse], dim=1)
                refined_logits = gen(g_input)
                refined_probs = torch.sigmoid(refined_logits)
                
                d_input_fake = torch.cat([real_ct, refined_probs.detach()], dim=1)
                d_fake = disc(d_input_fake)
                loss_d_fake = criterion_GAN(d_fake, torch.zeros_like(d_fake))
                
                loss_d = (loss_d_real + loss_d_fake) * 0.5

            scaler.scale(loss_d).backward()
            scaler.step(opt_d)

            # --- 2. Generator Step ---
            opt_g.zero_grad()
            with autocast(device_type=device.type):
                # Adversarial (Fool D)
                # Note: We re-calculate d_fake WITH gradients flowing to G
                d_input_g = torch.cat([real_ct, torch.sigmoid(refined_logits)], dim=1)
                d_pred = disc(d_input_g)
                loss_g_adv = criterion_GAN(d_pred, torch.ones_like(d_pred))
                
                # Segmentation (Structure)
                loss_g_seg = criterion_BCE(refined_logits, real_gt) + criterion_Tversky(refined_logits, real_gt)
                
                loss_g = (LAMBDA_SEG * loss_g_seg) + (LAMBDA_ADV * loss_g_adv)

            scaler.scale(loss_g).backward()
            scaler.step(opt_g)
            scaler.update()

            print(f"Step {step+1}: D Loss={loss_d.item():.4f}, G Loss={loss_g.item():.4f}")
            
            if step == 0:
                initial_loss = loss_g.item()
            elif step == 2:
                if loss_g.item() != initial_loss:
                    print("\nSUCCESS: Loss is changing. Gradients are flowing.")
                else:
                    print("\nWARNING: Loss is identical. Check for detached gradients.")

if __name__ == "__main__":
    run_test()

Running test on: cuda
Setup complete. Starting training loop...
D Input: torch.Size([2, 2, 64, 64, 64])
D Layer 1: torch.Size([2, 16, 32, 32, 32])
D Layer 2: torch.Size([2, 32, 16, 16, 16])
D Layer 3: torch.Size([2, 64, 8, 8, 8])
D Layer 4: torch.Size([2, 128, 7, 7, 7])
D Output (Patch Grid): torch.Size([2, 1, 6, 6, 6])
D Input: torch.Size([2, 2, 64, 64, 64])
D Layer 1: torch.Size([2, 16, 32, 32, 32])
D Layer 2: torch.Size([2, 32, 16, 16, 16])
D Layer 3: torch.Size([2, 64, 8, 8, 8])
D Layer 4: torch.Size([2, 128, 7, 7, 7])
D Output (Patch Grid): torch.Size([2, 1, 6, 6, 6])
D Input: torch.Size([2, 2, 64, 64, 64])
D Layer 1: torch.Size([2, 16, 32, 32, 32])
D Layer 2: torch.Size([2, 32, 16, 16, 16])
D Layer 3: torch.Size([2, 64, 8, 8, 8])
D Layer 4: torch.Size([2, 128, 7, 7, 7])
D Output (Patch Grid): torch.Size([2, 1, 6, 6, 6])
Step 1: D Loss=0.4743, G Loss=137.0488
D Input: torch.Size([2, 2, 64, 64, 64])
D Layer 1: torch.Size([2, 16, 32, 32, 32])
D Layer 2: torch.Size([2, 32, 16, 16, 16

### Building the training class

In [3]:
import json
import sys
from os.path import join
sys.path.append("../utils")
from main_train_class import main_train_STU_Net
from GANs_networks import Generator, PatchDiscriminator
from tqdm import tqdm
from torch.nn.functional import sigmoid, binary_cross_entropy_with_logits
# Standard Library Imports
from os.path import join
import sys

import json

# Third-Party Library Imports
import torch
import torch.optim as optim
from torch.nn.functional import sigmoid
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.amp import autocast, GradScaler
from torch.nn import MSELoss
# MONAI Specific Imports
import monai
from monai.data import CacheDataset

from monai.transforms import (
    Compose,
    CopyItemsd,
    LoadImaged, 
    ScaleIntensityRanged, 
    ResizeWithPadOrCropd,
    EnsureTyped,
    EnsureChannelFirstd
)
from mask_utils import GetROIMaskdd, GetBinaryLabeld
# Local Project Imports

class postprocessGANs(main_train_STU_Net):
    def __init__(self, config):
        # TODO predict and save logits from the segmentation network
        self.config = config
        self.labda_seg = self.config['labda_seg']
        self.G_model, self.D_model = self._build_models()
        self.train_loader = self._set_train_dataloader() 
        self.val_loader = self._set_val_dataloader() 
        self.opt_G, self.opt_D = self._set_optimizers()
        self.wandb_run = self._set_wandb_checkpoint() # Heritage
        # Expects lists of predictions # TODO put predictions inside of a list
        self.G_voxel_criterion = self._set_train_criterion() # Heritage
        self.criterion_GAN = MSELoss()
        self.val_metric = self._set_val_metric() # Heritage
        self.G_cosAnnealLR, self.D_cosAnnealLR = self._set_scheduler()

        # set scaler for mix precision 
        self.G_scaler = GradScaler()
        self.D_scaler = GradScaler()
        
        # check resume
        self._resume()

    def _build_models(self):
        # TODO change to 16 the first_channels and initial_filters
        gen = Generator(in_channels=2, first_channels=16, out_channels=1, use_checkpointing=True).to(self.config['device'])
        disc = PatchDiscriminator(in_channels=2, initial_filters=8).to(self.config['device'])
        return gen, disc
    
    def _resume(self):
        if self.config.get('D_resume'):
            # Dis
            D_checkpoint = torch.load(self.config['D_resume'], map_location="cpu", weights_only=False) 
            D_model_weights = D_checkpoint['model_weights']  
            self.D_model.load_state_dict(D_model_weights, strict=True)
            self.D_model = self.D_model.to(self.config['device'])
            # optimizer load
            self.opt_D.load_state_dict(D_checkpoint['optimizer_state_dict'])
        if self.config.get('G_resume'):
            G_checkpoint = torch.load(self.config['G_resume'], map_location="cpu", weights_only=False) 
            G_model_weights = G_checkpoint['model_weights']  
            self.G_model.load_state_dict(G_model_weights, strict=True)
            self.G_model = self.G_model.to(self.config['device'])
            # optimizer load
            self.opt_G.load_state_dict(G_checkpoint['optimizer_state_dict'])
            # parameters
            self.start_epoch = G_checkpoint['epoch'] + 1 # To continue to the next epoch instead of repeating  
            self.val_value = G_checkpoint['val_value']
        else:
            self.start_epoch = 0
            self.val_value = 0

    def _set_train_dataloader(self):
        """ Getting the list of cases for training and loading using MONAI (all into memory)"""
        data_list = []
        # TODO: change back to this data list
        #with open(self.config['data_split_json'], "r") as f:
        #    split = json.load(f)
        #train_cases = split["train"]
        #
        train_cases = ["2290837.nii.gz"]
        for train_case in train_cases:
            complete_data_dict = {}
            complete_data_dict["image"] = join(self.config['vol_data_path'], train_case)
            complete_data_dict["gt"] = join(self.config['label_data_path'], train_case)
            complete_data_dict["bridge_weight_map"] = join(self.config['bridge_weight_map_path'], train_case)
            # TODO in the json the pred_seg_logits needs to be changed!
            complete_data_dict["pred_seg_logits"] = join(self.config['pred_seg_logits'], train_case)
            data_list.append(complete_data_dict)
            
            if self.config['debug']:
                for i in range(30):
                    data_list.append(complete_data_dict)
                print(f"training using case: {data_list[0]}")
                break  # repeat 30 cases for debug mode

        print(f"Train cases: {len(train_cases)}")
        print(f"Some examples:")
        print(train_cases[:5])

        transforms_list = [   
                # Load image 
                LoadImaged(keys=["image", 'gt', 'bridge_weight_map', 'pred_seg_logits']),
                EnsureChannelFirstd(keys=["image", 'gt', 'bridge_weight_map', 'pred_seg_logits']),

                # Normalize uint8 input
                ScaleIntensityRanged(keys=["image"], a_min=0, a_max=255, b_min=0, b_max=1, clip=True),

                # Create a ROI mask for cropping 
                GetROIMaskdd(keys=["gt"], ignore_mask_value=2, new_key_names=["roi_mask"]),

                # Cropping or padding if bigger or smaller (expected to be all equall of smaller)
                ResizeWithPadOrCropd(keys=["image", 'gt', 'roi_mask', 'bridge_weight_map', 'pred_seg_logits'], spatial_size=self.config['patch_size'], mode="minimum"),
                GetBinaryLabeld(keys=["gt"], ignore_mask_value=2),
                EnsureTyped(keys=["image", "gt", "roi_mask", "bridge_weight_map", 'pred_seg_logits'], track_meta=False)
        ]

        transforms = Compose(transforms_list)
        
        print("Initializing Dataset...")
        train_ds = CacheDataset(
            data=data_list, 
            transform=transforms, 
            cache_rate=self.config['train_cache_rate'],  
            num_workers=self.config['num_workers'], 
            progress=True
        )

        print("Initializing Train DataLoader...")
        train_loader = monai.data.DataLoader(
            train_ds, 
            batch_size=self.config['batch_size'], 
            num_workers=self.config['num_workers'],
            shuffle=True,      
            pin_memory=True
        )
        return train_loader
    
    def _set_val_dataloader(self):
        data_list = []
        with open(self.config['data_split_json'], "r") as f:
            split = json.load(f)

        val_cases = split["val"]

        if self.config['debug']: 
            print(f"Debug mode: Using training cases for validation dataloader")
            # train_cases = split["train"] # TODO uncomment
            train_cases = ["2290837.nii.gz"] # TODO remove
            for train_case in train_cases:
                complete_data_dict = {}
                complete_data_dict["image"] = join(self.config['vol_data_path'], train_case)
                complete_data_dict["gt"] = join(self.config['label_data_path'], train_case)
                complete_data_dict["bridge_weight_map"] = join(self.config['bridge_weight_map_path'], train_case)
                complete_data_dict["pred_seg_logits"] = join(self.config['pred_seg_logits'], train_case)
                data_list.append(complete_data_dict)
                print(f"Validation using case: {train_case}")
                break  # The same training sample for validation in debug mode
        else:
            for val_case in val_cases:
                complete_data_dict = {}
                complete_data_dict["image"] = join(self.config['vol_data_path'], val_case)
                complete_data_dict["gt"] = join(self.config['label_data_path'], val_case)
                complete_data_dict["bridge_weight_map"] = join(self.config['bridge_weight_map_path'], val_case)
                complete_data_dict["pred_seg_logits"] = join(self.config['pred_seg_logits'], val_case)
                data_list.append(complete_data_dict)

        print(f"Val cases: {len(val_cases)}")
        print(f"Some examples:")
        print(val_cases[:5])

        transforms = Compose(
            [   
                # Load image 
                LoadImaged(keys=["image", 'gt', 'bridge_weight_map', 'pred_seg_logits']),
                EnsureChannelFirstd(keys=["image", 'gt', 'bridge_weight_map', 'pred_seg_logits']),
                # Normalize uint8 input
                ScaleIntensityRanged(keys=["image"], a_min=0, a_max=255, b_min=0, b_max=1, clip=True),
                # Create a ROI mask for cropping 
                GetROIMaskdd(keys=["gt"], ignore_mask_value=2, new_key_names=["roi_mask"]),
                # Get random patches
                ResizeWithPadOrCropd(keys=["image", "gt", "roi_mask", 'bridge_weight_map', 'pred_seg_logits'], spatial_size=self.config['patch_size'], mode="minimum"),
                GetBinaryLabeld(keys=["gt"], ignore_mask_value=2),
                EnsureTyped(keys=["image", "gt", "roi_mask", 'bridge_weight_map', 'pred_seg_logits'], track_meta=False)
            ]
        )

        print("Initializing Dataset...")
        val_ds = CacheDataset(
            data=data_list, 
            transform=transforms, 
            cache_rate=self.config['val_cache_rate'],  
            num_workers=self.config['num_workers'], 
            progress=True
        )
        
        print("Initializing Val DataLoader...")
        val_loader = monai.data.DataLoader(val_ds, batch_size=1, num_workers=self.config['num_workers'])
        return val_loader

    def _set_optimizers(self):
        """Define the optimizer (e.g., Adam, SGD)."""
        opt_g = optim.AdamW(self.G_model.parameters(), lr=self.config['learning_rate_G'])
        opt_d = optim.AdamW(self.D_model.parameters(), lr=self.config['learning_rate_D'])
        return opt_g, opt_d
    
    def _set_scheduler(self):
        """Define learning rate scheduler."""
        # If resuming, last_epoch should be start_epoch - 1
        last_epoch = self.config.get('resume_epoch', 0) - 1 if self.config.get('resume') else -1
        G_cosAnnealLR = CosineAnnealingLR(self.opt_G, self.config['num_epochs'], eta_min=0.0, last_epoch=last_epoch)
        D_cosAnnealLR = CosineAnnealingLR(self.opt_D, self.config['num_epochs'], eta_min=0.0, last_epoch=last_epoch)
        return G_cosAnnealLR, D_cosAnnealLR
    
    def saving_logic(self, best_val_value, val_avg_value, epoch):
        """ Logic to save the best model and periodic checkpoints """

        self.G_model, self.D_model

        if best_val_value < val_avg_value: 
            best_val_value = val_avg_value
            # Dis
            D_save_path = join(self.model_save_path, f"D_model_best.pth")
            torch.save({
                    'epoch': epoch,
                    'model_weights': self.D_model.state_dict(),  
                    'optimizer_state_dict': self.opt_D.state_dict(),
                    'val_value': val_avg_value,
                }, D_save_path)
            print(f"Saved checkpoint: {D_save_path}")
            # Gen
            G_save_path = join(self.model_save_path, f"G_model_best.pth")
            torch.save({
                    'epoch': epoch,
                    'model_weights': self.G_model.state_dict(),  
                    'optimizer_state_dict': self.opt_G.state_dict(),
                    'val_value': val_avg_value,
                }, G_save_path)
            print(f"Saved checkpoint: {G_save_path}")
        
        # Save Checkpoint
        if epoch % 10 == 0: 
            # Dis
            D_save_path = join(self.model_save_path, f"D_model_epoch_{epoch}.pth")
            torch.save({
                    'epoch': epoch,
                    'model_weights': self.D_model.state_dict(),  
                    'optimizer_state_dict': self.opt_D.state_dict(),
                    'val_value': val_avg_value,
                }, D_save_path)
            print(f"Saved checkpoint: {D_save_path}")
            # Gen
            G_save_path = join(self.model_save_path, f"G_model_epoch_{epoch}.pth")
            torch.save({
                    'epoch': epoch,
                    'model_weights': self.G_model.state_dict(),  
                    'optimizer_state_dict': self.opt_G.state_dict(),
                    'val_value': val_avg_value,
                }, G_save_path)
            print(f"Saved checkpoint: {G_save_path}")
        return best_val_value

    def train_epoch(self, **kwargs):
        """Logic for a single training epoch. Returns average loss."""
        epoch = kwargs.get('epoch')
        warmup_G = kwargs.get('warmup_G')
        
        D_epoch_loss = 0
        G_epoch_loss = 0

        D_per_criterio_loss = {
            'loss_d_real' : 0,
            'loss_d_fake' : 0
        }
        G_per_criterio_loss = {} # voxel wise metrics dict

        self.G_model.train()
        self.D_model.train()

        pbar = tqdm(self.train_loader, desc=f"Epoch {epoch}/{self.config['num_epochs']}")

        for idx, batch_dict in enumerate(pbar):
            # batch_dict contains:
                # image (volume normalized)
                # gt (the real ground truth binary)
                # the roi_mask (with the region to ignore 0 and to consider 1)
                # weighted map for the bridge weight loss
                # segmentation logits (pre-computed to save time!)
            input_image = batch_dict['image'].to(self.config['device'])
            ground_truth = batch_dict['gt'].to(self.config['device'])
            roi_mask = batch_dict['roi_mask'].to(self.config['device'])
            bridge_weight_map = batch_dict['bridge_weight_map'].to(self.config['device'])
            pred_seg_logits = batch_dict['pred_seg_logits'].to(self.config['device']) # TODO change to pred_seg_logits

            if not warmup_G:
                #################################################
                #  UPDATE DISCRIMINATOR
                #################################################
                self.opt_D.zero_grad()

                # --- FP16 FORWARD PASS ---
                with autocast(device_type=self.config['device']):
                    # --- Train with REAL (CT + Ground Truth) ---
                    # adding noise to the gt seg to avoid D to easily distinguish between real and fake
                    noisy_real_gt = ground_truth + (torch.randn_like(ground_truth) * 0.05) 
                    input_D = torch.cat([input_image, noisy_real_gt], dim=1)
                    prediction_D_real = self.D_model(input_D) # the prediction has the deep supervision! 3 outputs

                    # Target for REAL is 1.0
                    target_real = torch.ones_like(prediction_D_real) * 0.9 # we don't want overconfident predictions!
                    loss_d_real = self.criterion_GAN(prediction_D_real, target_real)
                    
                    # --- Train with FAKE (CT + Refined Prediction) ---
                    # Generate Fake
                    input_G = torch.cat([input_image, pred_seg_logits], dim=1)
                    with torch.no_grad():
                        refined_logits = self.G_model(input_G)
                        refined_probs = torch.sigmoid(refined_logits) # Convert logits to prob for D
                    # adding noise here as well, as explained before
                    noisy_fake_probs = refined_probs.detach() + (torch.randn_like(refined_probs) * 0.05)
                    
                    # Detach! Crucial! 
                    # We don't want to backprop into Generator yet.
                    fake_pair = torch.cat([input_image, noisy_fake_probs], dim=1)
                    prediction_D_fake = self.D_model(fake_pair)
                    
                    # Target for FAKE is 0.0
                    target_fake = torch.zeros_like(prediction_D_fake)
                    loss_d_fake = self.criterion_GAN(prediction_D_fake, target_fake)
                    
                    # Combine
                    loss_d = (loss_d_real + loss_d_fake) * 0.5

                # Backward D
                self.D_scaler.scale(loss_d).backward()
                self.D_scaler.step(self.opt_D)
                # Update Scaler once per batch
                self.D_scaler.update()

            #################################################
            #  UPDATE GENERATOR
            #  Goal: Fool the Discriminator (D outputs 1 for fake)
            #        AND minimize segmentation error
            #################################################
            self.opt_G.zero_grad()
            
            with autocast(device_type=self.config['device']):
                # Adversarial Loss (Fool D)
                # Construct the pair again (this time WITH gradients)
                # Use probabilities for D input
                input_G = torch.cat([input_image, pred_seg_logits], dim=1)
                refined_logits = self.G_model(input_G)
                refined_probs_for_g = torch.sigmoid(refined_logits) 
                
                # Segmentation Loss (Ground Truth Accuracy)
                # Compare refined_logits directly to real_gt_mask
                voxel_wise_loss, G_losses_dict = self.G_voxel_criterion([refined_logits], [ground_truth], roi_mask=[roi_mask], bridge_weight_map=[bridge_weight_map]) 
                
                if not warmup_G:
                    # Do Dis step for Gen
                    input_D_for_G = torch.cat([input_image, refined_probs_for_g], dim=1)
                    pred_D_for_G = self.D_model(input_D_for_G)
                    # Generator wants D to say "1.0" (Real)
                    target_G = torch.ones_like(pred_D_for_G)
                    loss_g_adv = self.criterion_GAN(pred_D_for_G, target_G)
                    G_losses_dict['loss_g_adv'] = loss_g_adv.item() # to save it latter
                else:
                    G_losses_dict['loss_g_adv'] = 0.0
                # Combine
                # loss_seg dominates (weight 100), loss_adv refines
                loss_g_total = (self.labda_seg * voxel_wise_loss)
                if not warmup_G:
                    loss_g_total += loss_g_adv

            # Backward G
            self.G_scaler.scale(loss_g_total).backward()
            self.G_scaler.step(self.opt_G)
            
            # Update Scaler once per batch
            self.G_scaler.update()
            
            
            ##### Handle loss graphs #####
            if not warmup_G:
                # Dis
                # overall loss
                D_epoch_loss += loss_d.item()
                # Per real vs fake loss
                D_per_criterio_loss['loss_d_real'] += loss_d_real.item()
                D_per_criterio_loss['loss_d_fake'] += loss_d_fake.item()
            

            # Gen
            # Overall loss
            G_epoch_loss += loss_g_total.item()
            
            # adding all individual metrics to the G dict
            for G_criterio_name in G_losses_dict.keys():
                if G_criterio_name in G_per_criterio_loss:
                    G_per_criterio_loss[G_criterio_name] += G_losses_dict[G_criterio_name]
                else:
                    G_per_criterio_loss[G_criterio_name] = G_losses_dict[G_criterio_name]
            
            # Update status bar
            if not warmup_G:
                # Update status bar
                pbar.set_postfix({
                    "G_Loss": loss_g_total.item(), 
                    "D_Loss": loss_d.item()
                })
            else:
                pbar.set_postfix({
                    "G_Loss": loss_g_total.item()
                })
        
        if epoch%10 == 0:
            # Save a prediction
            self.save_vol(refined_probs_for_g, join(self.preds_path, f"epoch_{epoch}_pred_train.nii.gz"))
            self.save_vol(input_image, join(self.preds_path, f"epoch_{epoch}_input_train.nii.gz"))
            self.save_vol(pred_seg_logits, join(self.preds_path, f"epoch_{epoch}_input_pred_seg_logits_train.nii.gz"))
            self.save_vol(ground_truth, join(self.preds_path, f"epoch_{epoch}_gt_train.nii.gz"))

        D_train_avg_loss = D_epoch_loss / len(self.train_loader)
        G_train_avg_loss = G_epoch_loss / len(self.train_loader)
        print(f"Epoch {epoch} Finished. Avg D_Loss: {D_train_avg_loss:.6f} | Avg G_Loss: {G_train_avg_loss:.6f}")
        
        # This will replace each element in the dict with the mean
        for criterio_name in G_losses_dict.keys():
            G_per_criterio_loss[criterio_name] = G_per_criterio_loss[criterio_name] / len(self.train_loader)
        if not warmup_G:
            D_per_criterio_loss['loss_d_real'] = D_per_criterio_loss['loss_d_real'] / len(self.train_loader)
            D_per_criterio_loss['loss_d_fake'] = D_per_criterio_loss['loss_d_fake'] / len(self.train_loader)

        return D_train_avg_loss, G_train_avg_loss, D_per_criterio_loss, G_per_criterio_loss
    
    def val(self, **kwargs):
        """Logic for evaluation. Returns a dictionary of metrics."""
        epoch = kwargs.get('epoch')
        self.G_model.eval()
        self.D_model.eval()
        
        # General DSC validation value for quality controll
        val_value_sum = 0
        epoch_val_pixel_loss = 0
        adv_fake = 0
        adv_real = 0
        # Add the per criterio val loss for checking overfitting
        per_criterio_val_loss = {}
        for val_criterio_name in self.config['criterion']:
            per_criterio_val_loss[f"val_{val_criterio_name}"] = 0

        pbar = tqdm(self.val_loader, desc=f"Val epoch {epoch}/{self.config['num_epochs']}")
        for idx, batch_dict in enumerate(pbar):
            input_image = batch_dict['image'].to(self.config['device'])
            ground_truth = batch_dict['gt'].to(self.config['device']) 
            # Create the mask of the region to compute the loss
            roi_mask = batch_dict['roi_mask'].to(self.config['device'])
            bridge_weight_map = batch_dict['bridge_weight_map'].to(self.config['device'])
            pred_seg_logits = batch_dict['pred_seg_logits'].to(self.config['device'])

            with torch.no_grad():
                # Forward Pass
                # --- REAL (CT + Ground Truth) ---
                input_D = torch.cat([input_image, ground_truth], dim=1)
                prediction_D_real = self.D_model(input_D) # the prediction has the deep supervision! 3 outputs
                # Target for REAL is 1.0
                target_real = torch.ones_like(prediction_D_real)
                loss_d_real = self.criterion_GAN(prediction_D_real, target_real)
                
                # --- FAKE (CT + Refined Prediction) ---
                # Generate Fake
                input_G = torch.cat([input_image, pred_seg_logits], dim=1)
                refined_logits = self.G_model(input_G)
                refined_probs = torch.sigmoid(refined_logits) # Convert logits to prob for D
                fake_pair = torch.cat([input_image, refined_probs.detach()], dim=1)
                prediction_D_fake = self.D_model(fake_pair)
                
                # Target for FAKE is 0.0
                target_fake = torch.zeros_like(prediction_D_fake)
                loss_d_fake = self.criterion_GAN(prediction_D_fake, target_fake)

                # Calculate DSC (Compare Prediction vs. GT)
                val_value = self.val_metric(pred=refined_logits, target=ground_truth, roi_mask=roi_mask)
                # Also compute val losses for logging (no deep supervision here)
                val_loss, val_losses_dict = self.G_voxel_criterion([refined_logits], [ground_truth], roi_mask=[roi_mask], bridge_weight_map=[bridge_weight_map], deep_supervision_weights=[1.0]) 
                # commented to avoid overwhelming 
                #self.wandb_run.log({"val_value": val_value.item()})

            val_value_sum += val_value # val metric (DSC)
            epoch_val_pixel_loss += val_loss.item() # all losses function used for training (except Adv)
            adv_fake += loss_d_fake.item()
            adv_real += loss_d_real.item()
            for val_criterio_name in val_losses_dict.keys():
                per_criterio_val_loss[f"val_{val_criterio_name}"] += val_losses_dict[f"{val_criterio_name}"]
            pbar.set_postfix({"DSC": val_value})

        if epoch%10 == 0:
            pred_save = sigmoid(refined_logits)
            pred_save[pred_save>0.5] = 1.0
            pred_save[pred_save<=0.5] = 0.0
            self.save_vol(refined_logits, join(self.preds_path, f"epoch_{epoch}_logits_val.nii.gz"))
            self.save_vol(pred_save, join(self.preds_path, f"epoch_{epoch}_pred_val.nii.gz"))
            self.save_vol(input_image, join(self.preds_path, f"epoch_{epoch}_input_val.nii.gz"))
            self.save_vol(pred_seg_logits, join(self.preds_path, f"epoch_{epoch}_input_pred_seg_logits_val.nii.gz"))
            self.save_vol(ground_truth, join(self.preds_path, f"epoch_{epoch}_gt_val.nii.gz"))

        # computing mean of metrics
        val_avg_value = val_value_sum / len(self.val_loader)
        val_avg_pixel_loss = epoch_val_pixel_loss / len(self.val_loader)

        for val_criterio_name in val_losses_dict.keys():
            per_criterio_val_loss[f"val_{val_criterio_name}"] = per_criterio_val_loss[f"val_{val_criterio_name}"] / len(self.val_loader)
        per_criterio_val_loss["val_adv_fake"] = adv_fake / len(self.val_loader)
        per_criterio_val_loss["val_adv_real"] = adv_real / len(self.val_loader)
        print(f"Epoch {epoch} with validation avg DSC: {val_avg_value:.6f} | avg Loss: {val_avg_pixel_loss:.6f} | adv fake loss {per_criterio_val_loss['val_adv_fake']} | adv real loss {per_criterio_val_loss['val_adv_real']}")
        return val_avg_value, val_avg_pixel_loss, per_criterio_val_loss 
    
    def train_loop(self, **kwargs):
        """Standardized training loop."""
        best_val_value = self.val_value
        
        # Make sure all weights are trainable
        for param in self.G_model.parameters():
            param.requires_grad = True
        for param in self.D_model.parameters():
            param.requires_grad = True

        for self.epoch in range(self.start_epoch, self.config['num_epochs'] + 1):
            if self.epoch<1000: # TODO maybe put in 10
                warmup_G = True
            else:
                warmup_G = False
            # Train one epoch
            D_train_avg_loss, G_train_avg_loss, D_per_criterio_loss, G_per_criterio_loss = self.train_epoch(
                epoch=self.epoch,
                warmup_G=warmup_G
            )
            # Perform evaluation 
            val_avg_value, val_avg_pixel_loss, per_criterio_val_loss  = self.val(
                epoch=self.epoch
            )

            # Save in wandb
            log_train_data = {
                    "epoch_train": self.epoch,
                    "D_train_avg_loss": D_train_avg_loss,
                    "G_train_avg_loss": G_train_avg_loss,
                    "val_Dice": val_avg_value,
                    "val_avg_pixel_loss": val_avg_pixel_loss,
                    "G_lr": self.opt_G.param_groups[0]['lr'],
                    "D_lr": self.opt_D.param_groups[0]['lr']  
                }
            # Add the per criterio losses to wandb
            for criterio_name in D_per_criterio_loss.keys():
                log_train_data[criterio_name] = D_per_criterio_loss[criterio_name]

            for criterio_name in G_per_criterio_loss.keys():
                log_train_data[criterio_name] = G_per_criterio_loss[criterio_name]
                if criterio_name.endswith("_fullres"):
                    print(f"_fullres is still in the loss function! It should not!")


            for val_criterio_name in per_criterio_val_loss.keys():
                log_train_data[val_criterio_name] = per_criterio_val_loss[val_criterio_name]

            self.wandb_run.log(
                log_train_data
            )

            # Checking if saving 
            best_val_value = self.saving_logic(
                best_val_value=best_val_value, 
                val_avg_value=val_avg_value, 
                epoch=self.epoch
            )

            # Applying learning rate Cosine Annealing
            self.G_cosAnnealLR.step() 
            self.D_cosAnnealLR.step()

In [ ]:
CONFIG_FILE = '../configs/post_process_GANs.json'
with open(CONFIG_FILE, "r") as f:
    config_content = json.load(f)
GANs_train_object = postprocessGANs(config_content)
GANs_train_object.train_loop()

training using case: {'image': '/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/small_train/images/2290837.nii.gz', 'gt': '/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/small_train/labels/2290837.nii.gz', 'bridge_weight_map': '/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/small_train/train_bridge_weight_map/2290837.nii.gz', 'pred_seg_logits': '/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/small_train/pred_seg_logits/2290837.nii.gz'}
Train cases: 1
Some examples:
['2290837.nii.gz']
Initializing Dataset...
Initializing Train DataLoader...
Debug mode: Using training cases for validation dataloader
Validation using case: 2290837.nii.gz
Val cases: 79
Some examples:
['693501383.nii.gz', '118041886.nii.gz', '571334887.nii.gz', '2536049117.nii.gz', '1127903126.nii.gz']
Initializing Dataset...


Loading dataset:   0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 1/1 [00:02<00:00,  2.30s/it]


Initializing Val DataLoader...


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


...

Epoch 258/1000: 100%|██████████| 31/31 [00:41<00:00,  1.33s/it, G_Loss=0.105]


Epoch 258 Finished. Avg D_Loss: 0.000000 | Avg G_Loss: 0.105116


Val epoch 258/1000: 100%|██████████| 1/1 [00:00<00:00,  1.51it/s, DSC=0.781]


Epoch 258 with validation avg DSC: 0.780828 | avg Loss: 0.420316 | adv fake loss 0.15130695700645447 | adv real loss 1.1278865337371826
Saved checkpoint: ../checkpoints/post_processing_step/wandb/run-20260121_214229-n6391o6b/files/model/D_model_best.pth
Saved checkpoint: ../checkpoints/post_processing_step/wandb/run-20260121_214229-n6391o6b/files/model/G_model_best.pth


Epoch 259/1000:  35%|███▌      | 11/31 [00:18<00:25,  1.27s/it, G_Loss=0.105]

In [ ]:
# TODO check if the D_loss is within the value expected